In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import base64
import os
import re

from animal_shelter import AnimalShelter

In [2]:
###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(
    user='accuser',
    password='SNHU1234',
    host='nv-desktop-services.apporto.com',
    port=32592,
    db='aac',
    collection='animals'
)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

# Convert 'rec_num' to string for filtering
df['rec_num'] = df['rec_num'].astype(str)

# Construct the file path using the $HOME environment variable
home_path = os.path.expanduser('~')
image_path = os.path.join(home_path, 'ProjectTwo', 'assets', 'GraziosoSalvareLogo.png')

# Read the image and convert it to base64 using the dynamic path
encoded_image = base64.b64encode(open(image_path, 'rb').read()).decode('ascii')

In [3]:
#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    # Branding
    html.A(
        href='https://www.snhu.edu',
        children=html.Img(src='data:image/png;base64,{}'.format(encoded_image), height='100px'),
        target='_blank'
    ),
    html.Center(html.B(html.H1('Rescue to Rescuer'))),
    html.Hr(),
    
    # Filter options
    html.Div([
        html.Label('Filter by Rescue Type:'),
        dcc.RadioItems(
            id='filter-radio',
            options=[
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'block'}  # Display each option on a new line
        )
    ]),
    html.Br(),
    
    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        sort_action='native',
        filter_action='native',
        page_size=10,
        row_selectable='multi',
        selected_rows=[],
        style_table={'overflowX': 'auto'},
        style_cell={
            'height': 'auto',
            'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
            'whiteSpace': 'normal'
        },
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        style_data_conditional=[{
            'if': {'column_id': 'column_to_highlight'},
            'backgroundColor': 'rgb(255, 230, 230)',
            'color': 'black'
        }]
    ),
    html.Br(),
    html.Hr(),
    html.Div(
        id='map-id',
        className='col s12 m6',
    ),
    html.Br(),
    html.Div(
        id='pie-chart-id',
        className='col s12 m6',
    ),
    html.Footer('Developed by Joseph Les')
])

In [4]:
#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(
    Output('datatable-id', 'data'),
    Input('filter-radio', 'value')
)
def update_table(filter_value):
    # Define breed filters based on rescue type
    rescue_breeds = {
        'Water': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland Mix'],
        'Mountain': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'],
        'Disaster': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']
    }
    
    if filter_value == 'Reset':
        filtered_df = df
    else:
        # Use the corresponding breed list for the selected rescue type
        filtered_df = df[df['breed'].str.contains('|'.join(map(re.escape, rescue_breeds[filter_value])), na=False)]

    return filtered_df.to_dict('records')

@app.callback(
    Output('map-id', "children"),
    [Input('filter-radio', 'value'),
     Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(filter_value, viewData, indices):
    if viewData is None or len(viewData) == 0 or filter_value == 'Reset':
        return [dl.Map(style={'width': '1000px', 'height': '500px'},
                       center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id")
        ])]

    dff = pd.DataFrame.from_dict(viewData)
    markers = []
    
    if indices is not None and len(indices) > 0:
        # Show only markers for selected rows
        dff = dff.iloc[indices]

    for row in range(len(dff)):
        try:
            lat = dff.iloc[row, 13]
            lon = dff.iloc[row, 14]

            # Check if lat and lon are valid numbers
            if pd.notna(lat) and pd.notna(lon):
                lat = float(lat)
                lon = float(lon)
                markers.append(
                    dl.Marker(position=[lat, lon],
                              children=[
                                  dl.Tooltip(dff.iloc[row, 4] if len(dff.columns) > 4 else "Unknown"),
                                  dl.Popup([
                                      html.H1("Animal Name"),
                                      html.P(dff.iloc[row, 9] if len(dff.columns) > 9 else "Unknown")
                                  ])
                              ])
                )
        except (IndexError, ValueError, TypeError):
            continue  # Skip rows with invalid data

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            *markers
        ])
    ]

@app.callback(
    Output('pie-chart-id', "children"),
    [Input('filter-radio', 'value'),
     Input('datatable-id', 'derived_virtual_data')]
)
def update_pie_chart(filter_value, viewData):
    if viewData is None or len(viewData) == 0:
        return html.Div("No data to display")

    dff = pd.DataFrame.from_dict(viewData)
    
    # Define breed filters based on rescue type
    rescue_breeds = {
        'Water': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland Mix'],
        'Mountain': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'],
        'Disaster': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']
    }

    if filter_value == 'Reset':
        # Calculate the percentage of each breed
        breed_counts = dff['breed'].value_counts(normalize=True)
        
        # Separate breeds with less than 2% into 'Other'
        other_threshold = 0.02
        main_breeds = breed_counts[breed_counts >= other_threshold]
        other_breeds = breed_counts[breed_counts < other_threshold]
        
        # Create a new DataFrame for the pie chart data
        pie_data = main_breeds.append(pd.Series(other_breeds.sum(), index=['Other'])).reset_index()
        pie_data.columns = ['breed', 'percentage']
    else:
        # Filter dff to only include the relevant breeds
        relevant_breeds = rescue_breeds.get(filter_value, [])
        dff = dff[dff['breed'].isin(relevant_breeds)]
        
        # Calculate the percentage of each breed for the filtered data
        pie_data = dff['breed'].value_counts(normalize=True).reset_index()
        pie_data.columns = ['breed', 'percentage']

    # Create the pie chart
    fig = px.pie(pie_data, values='percentage', names='breed', title='Distribution by Breed')
    
    return dcc.Graph(figure=fig)

app.run_server(debug=True)